<a href="https://colab.research.google.com/github/informatics-isi-edu/eye-ai-exec/blob/main/notebooks/VGG19_Diagnosis_Train.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# VGG19 Training --- Fine tuning on Van's Labels

This notebook is used to train VGG19 model for glacoma diagnosis.

In [ ]:
# import sys
# IN_COLAB = 'google.colab' in sys.modules

# if IN_COLAB:
#     !pip install deriva
#     !pip install bdbag
#     !pip install --upgrade --force pydantic
#     !pip install git+https://github.com/informatics-isi-edu/deriva-ml git+https://github.com/informatics-isi-edu/eye-ai-ml

In [ ]:
repo_dir = "Repos"   # Set this to be where your github repos are located.
%load_ext autoreload
%autoreload 2

# Update the load path so python can find modules for the model
import sys
from pathlib import Path
sys.path.insert(0, str(Path.home() / repo_dir / "eye-ai-ml"))

In [ ]:
# Prerequisites

import json
import os
from eye_ai.eye_ai import EyeAI
import pandas as pd
from pathlib import Path, PurePath
import logging
# import torch

logging.basicConfig(level=logging.INFO, format='%(asctime)s - %(levelname)s - %(message)s', force=True)

In [ ]:

from deriva.core.utils.globus_auth_utils import GlobusNativeLogin
catalog_id = "eye-ai" #@param
host = 'www.eye-ai.org'


gnl = GlobusNativeLogin(host=host)
if gnl.is_logged_in([host]):
    print("You are already logged in.")
else:
    gnl.login([host], no_local_server=True, no_browser=True, refresh_tokens=True, update_bdbag_keychain=True)
    print("Login Successful")

Connect to Eye-AI catalog.  Configure to store data local cache and working directories.  Initialize Eye-AI for pending execution based on the provided configuration file.

In [ ]:
# Variables to configure the rest of the notebook.

cache_dir = '/data'        # Directory in which to cache materialized BDBags for datasets
working_dir = '/data'    # Directory in which to place output files for later upload.

configuration_rid = "2-C94P" # rid
# Change the confi_file with bag_url=["minid: train", "minid: Valid", "minid: test"]


In [ ]:
EA = EyeAI(hostname = host, catalog_id = catalog_id, cache_dir= cache_dir, working_dir=working_dir)

In [ ]:
# @title Initiate an Execution
configuration_records = EA.execution_init(configuration_rid=configuration_rid)
configuration_records.model_dump()

In [ ]:
configuration_records

In [ ]:
exclude_train = pd.read_csv(configuration_records.assets_paths[1])['ID'].to_list()
exclude_valid = pd.read_csv(configuration_records.assets_paths[2])['ID'].to_list()

In [ ]:
# @title Data Preprocessing (Filtering Image.csv for just Field_2 Images)
train_dir = configuration_records.bag_paths[2] # path to the raw train dataset
validation_dir = configuration_records.bag_paths[3]
test_dir = configuration_records.bag_paths[4]

train_cropped_image_path, train_cropped_csv = EA.create_cropped_images(str(train_dir),
                                                                       output_dir = str(EA.working_dir) +'/train',
                                                                       crop_to_eye=True,
                                                                       exclude_list=exclude_train)
validation_cropped_image_path, validation_cropped_csv = EA.create_cropped_images(str(validation_dir),
                                                                                 output_dir = str(EA.working_dir) +'/valid',
                                                                                 crop_to_eye=True,
                                                                                 exclude_list=exclude_valid)
test_cropped_image_path, test_cropped_csv = EA.create_cropped_images(str(test_dir),
                                                                     output_dir = str(EA.working_dir) +'/test',
                                                                     crop_to_eye=True)


In [ ]:
# # without no optic disc images

import os

def count_files(directory):
    return len([name for name in os.listdir(directory) if os.path.isfile(os.path.join(directory, name))])

def analyze_directory(base_path):
    main_folders = ['train', 'test', 'valid']
    
    for main_folder in main_folders:
        main_folder_path = os.path.join(base_path, main_folder)
        if not os.path.exists(main_folder_path):
            print(f"{main_folder} folder not found")
            continue
        
        print(f"\nAnalyzing {main_folder} folder:")
        
        image_cropped_path = os.path.join(main_folder_path, 'Image_cropped')
        if not os.path.exists(image_cropped_path):
            print("Image_cropped folder not found")
            continue
        
        total_files = 0
        for subfolder in os.listdir(image_cropped_path):
            subfolder_path = os.path.join(image_cropped_path, subfolder)
            if os.path.isdir(subfolder_path):
                file_count = count_files(subfolder_path)
                print(f"  {subfolder}: {file_count} files")
                total_files += file_count
        
        print(f"Total files in {main_folder}: {total_files}")

# Assuming you're running this script from the directory containing train, test, and valid folders
base_path = "/data/sreenidhi/EyeAI_working/" #os.getcwd()
analyze_directory(base_path)



In [ ]:

output_path = str(EA.working_dir) + "/Execution_Assets/" + configuration_records.vocabs['Execution_Asset_Type'][0].name
os.mkdir(output_path)

In [ ]:
output_path

In [ ]:
best_hyper_parameters_json_path = str(configuration_records.assets_paths[0])

In [ ]:
best_hyper_parameters_json_path

In [ ]:
# @title Execute Training algorithm
from eye_ai.models.vgg19_diagnosis_fine_tune_train import main
with EA.execution(execution_rid=configuration_records.execution_rid) as exec:
  main(train_path=train_cropped_image_path,
       valid_path=validation_cropped_image_path, 
       test_path=test_cropped_image_path, 
       output_path = output_path,
       best_hyperparameters_json_path = best_hyper_parameters_json_path,
       model_name = "VGG19_Catalog_LAC_DHS_Cropped_Data_exlcuding_no_Optic_disc_fundus_Trained_model_June_24_2024_Van_Fine_Tuned",
       original_model_path = str(configuration_records.assets_paths[3])
       )
                    


In [ ]:
# @title Save Execution Assets (model) and Metadata
uploaded_assets = EA.execution_upload(configuration_records.execution_rid, True)